## 1. minio : data load

In [1]:
!echo '172.19.153.41    nlp-utils registry.nlp-utils minio.nlp-utils' | tee -a /etc/hosts

172.19.153.41    nlp-utils registry.nlp-utils minio.nlp-utils


In [2]:
!cat /etc/hosts

# Kubernetes-managed hosts file.
127.0.0.1	localhost
::1	localhost ip6-localhost ip6-loopback
fe00::0	ip6-localnet
fe00::0	ip6-mcastprefix
fe00::1	ip6-allnodes
fe00::2	ip6-allrouters
10.1.0.166	handson-0
172.19.153.41    registry.nlp-utils minio.nlp-utils
172.19.153.41    nlp-utilsregistry.nlp-utils minio.nlp-utils
172.19.153.41    nlp-utils registry.nlp-utils minio.nlp-utils
172.19.153.41    nlp-utils registry.nlp-utils minio.nlp-utils


In [7]:
!mc alias list

cache    
  URL       : http://172.20.41.234:9000
  AccessKey : minio
  SecretKey : minio123
  API       : s3v4
  Path      : auto

gcs      
  URL       : https://storage.googleapis.com
  AccessKey : YOUR-ACCESS-KEY-HERE
  SecretKey : YOUR-SECRET-KEY-HERE
  API       : S3v2
  Path      : dns

local    
  URL       : http://localhost:9000
  AccessKey : 
  SecretKey : 
  API       : 
  Path      : auto

minio    
  URL       : http://minio-service.kubeflow.svc.cluster.local:9000
  AccessKey : minio
  SecretKey : minio123
  API       : s3v4
  Path      : auto

nlp-utils
  URL       : http://nlp-utils:32900
  AccessKey : k8s
  SecretKey : nlplab2020
  API       : s3v4
  Path      : auto

play     
  URL       : https://play.min.io
  AccessKey : Q3AM3UQ867SPQQA43P2F
  SecretKey : zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG
  API       : S3v4
  Path      : auto

s3       
  URL       : https://s3.amazonaws.com
  AccessKey : YOUR-ACCESS-KEY-HERE
  SecretKey : YOUR-SECRET-KEY-HERE
  API       : 

In [457]:
from minio import Minio
from minio.error import ResponseError

In [576]:
s3_endpoint = 'nlp-utils:32900'
minio_username = "k8s"
minio_key = "nlplab2020"

#minio_endpoint = "http://" + s3_endpoint
#minio_region = "glue"

In [577]:
minioClient = Minio(
    s3_endpoint,
    access_key=minio_username,
    secret_key=minio_key,
    secure=False
)

In [578]:
# fgut_object(bucket_name, source_object_name, target_file_name ...) 
try:
    minioClient.fget_object('glue', 'cola.tar', './cola.tar')
    !tar -xvf cola.tar
except ResponseError as err:
    print(err)

CoLA/
CoLA/test.tsv
CoLA/train.tsv
CoLA/original/
CoLA/original/tokenized/
CoLA/original/tokenized/out_of_domain_dev.tsv
CoLA/original/tokenized/in_domain_train.tsv
CoLA/original/tokenized/in_domain_dev.tsv
CoLA/original/raw/
CoLA/original/raw/out_of_domain_dev.tsv
CoLA/original/raw/in_domain_train.tsv
CoLA/original/raw/in_domain_dev.tsv
CoLA/dev.tsv


## 2. train

In [282]:
# gpu가 있다면 gpu를 사용하고, 없으면 cpu를 사용한다.
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [416]:
# CoLA 데이터 load
import pandas as pd

df = pd.read_csv("./CoLA/train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
df_dev = pd.read_csv("./CoLA/dev.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
df_test = pd.read_csv("./CoLA/test.tsv", delimiter='\t', header=None, names=['index', 'sentence'])

print('Number of training sentences: {:,}\n'.format(df.shape[0]))
print('Number of dev sentences: {:,}\n'.format(df_dev.shape[0]))
print('Number of test sentences: {:,}\n'.format(df_test.shape[0]))

#df.sample(10)
#df_dev.sample(10)
#df_test.sample(10)

Number of training sentences: 8,551

Number of test sentences: 1,064

Number of dev sentences: 1,043



In [315]:
"""
BERT 토크나이저를 불러옵니다.
토크나이저 역할 : 1.문장 분리 2.[CLS],[SEP] 토큰 3. 토큰 to ID 4. truncate but not padding!
"""
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [417]:
"""
토큰을 id로 변환한다.
"""
input_ids = []
dev_ids = []
test_ids = []

for sent in df['sentence']:
    encoded_sent = tokenizer.encode(
                    sent,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    # This function also supports truncation and conversion
                    # to pytorch tensors, but we need to do padding, so we
                    # can't use these features :( .
                    #max_length = 128,          # Truncate all sentences.
                    #return_tensors = 'pt',     # Return pytorch tensors.
                    )

    input_ids.append(encoded_sent)
    
for sent in df_dev['sentence']:
    encoded_sent = tokenizer.encode(
                    sent,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    # This function also supports truncation and conversion
                    # to pytorch tensors, but we need to do padding, so we
                    # can't use these features :( .
                    #max_length = 128,          # Truncate all sentences.
                    #return_tensors = 'pt',     # Return pytorch tensors.
                    )

    dev_ids.append(encoded_sent)
    
for sent in df_test['sentence']:
    encoded_sent = tokenizer.encode(
                    sent,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    # This function also supports truncation and conversion
                    # to pytorch tensors, but we need to do padding, so we
                    # can't use these features :( .
                    #max_length = 128,          # Truncate all sentences.
                    #return_tensors = 'pt',     # Return pytorch tensors.
                    )

    test_ids.append(encoded_sent)

In [419]:
# 가장 긴 토큰 조회
print('trindataset Max sentence length: ', max([len(sen) for sen in input_ids]))
print('testdataset Max sentence length: ', max([len(sen) for sen in dev_ids]))
print('devdataset Max sentence length: ', max([len(sen) for sen in test_ids]))

trindataset Max sentence length:  47
testdataset Max sentence length:  35
devdataset Max sentence length:  38


In [420]:
"""
패딩 및 절삭
MAX_LEN 보다 작으면 문장 뒤쪽에(post) 0으로 패딩을 하고, 길면 절삭한다.
"""
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 64
print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

input_ids = pad_sequences(input_ids, 
                          maxlen=MAX_LEN, 
                          dtype="long", 
                          value=0, 
                          truncating="post",
                          padding="post")

dev_ids = pad_sequences(dev_ids, 
                          maxlen=MAX_LEN, 
                          dtype="long", 
                          value=0, 
                          truncating="post",
                          padding="post")


test_ids = pad_sequences(test_ids, 
                          maxlen=MAX_LEN, 
                          dtype="long", 
                          value=0, 
                          truncating="post",
                          padding="post")

print('Done.')


Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0
Done.


In [421]:
"""
어텐션 마스크를 생성한다.
패딩이면 
"""
attention_masks = []
dev_masks = []
test_masks = []

for sent in input_ids:

    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)
    
for sent in dev_ids:

    att_mask = [int(token_id > 0) for token_id in sent]
    dev_masks.append(att_mask)
    
for sent in test_ids:

    att_mask = [int(token_id > 0) for token_id in sent]
    test_masks.append(att_mask)

In [ ]:
"""
dev, test 셋으로 나눈다.
이미 데이터가 나눠져 있다면 수행하지 않아도 된다.
random state를 위 아래 동일하게 
"""
'''
from sklearn.model_selection import train_test_split
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)
'''

In [424]:
train_labels = []
dev_labels = []
test_labels = []

for label in df['label']:
    train_labels.append([label])
    
for label in df_dev['label']:
    dev_labels.append([label])
    
#for label in df_test['label']:
#    test_labels.append([label])

In [480]:
import numpy as np

# def list_to_float_torchtensor(your_list):
#     your_list = np.array(your_list, dtype=np.float64)
#     your_list = torch.from_numpy(your_list)
#     return your_list

def list_to_int_torchtensor(your_list):
    your_list = np.array(your_list, dtype=np.int64)
    your_list = torch.from_numpy(your_list)
    return your_list

In [489]:
train_inputs = list_to_int_torchtensor(input_ids)
train_masks = list_to_int_torchtensor(attention_masks)
train_labels = list_to_int_torchtensor(train_labels)

validation_inputs = list_to_int_torchtensor(dev_ids)
validation_masks = list_to_int_torchtensor(dev_masks)
validation_labels = list_to_int_torchtensor(dev_labels)

test_ids = list_to_int_torchtensor(test_ids)
test_masks = list_to_int_torchtensor(test_masks)

## 3. data loader

In [490]:
"""
pytorch의 dataloader를 사용한다.
배치사이즈
"""
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [491]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
            "bert-base-uncased",
            num_labels = 2,
            output_attentions = False,
            output_hidden_states = False,
)

#gpu를 사용할때만 cuda() 위로 올린다.
if torch.cuda.is_available():
    model.cuda()
else:
    model.cpu()

In [492]:
# 모델의 레이어별 학습 파라미터 차원을 확인한다.
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [493]:
from transformers import get_linear_schedule_with_warmup

epochs = 3

optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [494]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [495]:
import time
import datetime

def format_time(elapsed):

    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [496]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):
      
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_loss = 0
    
    model.train()
    
    # train 데이터 배치단위로 불러오기.
    for step, batch in enumerate(train_dataloader):
        
        # # Report progress : 진행사항은 매 40step마다 print한다.
        if step % 40 == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치 unpack 후 tensor를 gpu 혹은 cpu로 복사해야 한다.
        # 배치는 3가지 파이토치 텐서를 가지고 있다.
        # [0]: input ids 
        # [1]: attention masks
        # [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # backward 하기전에 항상 이전 계산된 그레디언트를 clear 해줘야 한다.(rnn은 그러면 안된다.)        
        model.zero_grad()

        # train data의 loss를 계산하기 위해 foward pass를 수행한다.
        outputs = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels)
        
        # ouputs type은 tuple인데 0번째 인자가 loss이다.
        loss = outputs[0]
        
        # 배치마다 loss를 모아두었다가 마지막 평균 loss를 계산한다.
        total_loss += loss.item()

        # gradients를 계산하기 위해 backward pass를 수행한다.
        loss.backward()
        
        # gradients발산을 막기위해 gradient cliping()을 한다.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # gradient를 고려하여 parameter를 update한다.
        optimizer.step()
        
        # warmup 고려하여 learning rate를 update한다.
        scheduler.step()
        
    # 1에폭 후 평균 loss를 계산한다.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # 러닝 커브를 plotting하기 위해 평균 loss를 저장한다.
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # Validation(dev) 데이터셋에 정확도를 측정한다.
    print("")
    print("Running Validation...")
    t0 = time.time()
    
    # 평가때는 eval() 모드로 전환해야 dropout layers가 작동하지 않는다.
    model.eval()
    
    # loss 및 acc 값 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    
    # 매 에폭마다 평가를 시행한다.
    for batch in validation_dataloader:
        
        # 배치를 GPU or CPU로 로드한다.
        batch = tuple(t.to(device) for t in batch)
        
        # dataloader를 uppack한다.
        b_input_ids, b_input_mask, b_labels = batch
        
        # 메모리 절약과 빠른 속도를 위해 gradient를 계산하지 않게 한다.
        with torch.no_grad():        
            
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # logits는 활성화함수(ex)softmax)를 통과하기 전 값이다.
        logits = outputs[0]
        
        # logits와 labels값을 cpu로 보낸다.
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 매 배치마다 정확도를 누적합 한다.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        
        # 배치수를 측정한다.
        nb_eval_steps += 1
    
    # 최종 결과를 보여준다.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
print("")
print("Training complete!")


======== Epoch 1 / 3 ========
Training...
  Batch    40  of    268.    Elapsed: 0:02:07.
  Batch    80  of    268.    Elapsed: 0:04:15.
  Batch   120  of    268.    Elapsed: 0:06:24.
  Batch   160  of    268.    Elapsed: 0:08:32.
  Batch   200  of    268.    Elapsed: 0:10:41.
  Batch   240  of    268.    Elapsed: 0:12:49.

  Average training loss: 0.50
  Training epcoh took: 0:14:17

Running Validation...
  Accuracy: 0.82
  Validation took: 0:00:30

======== Epoch 2 / 3 ========
Training...
  Batch    40  of    268.    Elapsed: 0:02:09.
  Batch    80  of    268.    Elapsed: 0:04:17.
  Batch   120  of    268.    Elapsed: 0:06:25.
  Batch   160  of    268.    Elapsed: 0:08:34.
  Batch   200  of    268.    Elapsed: 0:10:42.
  Batch   240  of    268.    Elapsed: 0:12:50.

  Average training loss: 0.32
  Training epcoh took: 0:14:18

Running Validation...
  Accuracy: 0.81
  Validation took: 0:00:30

======== Epoch 3 / 3 ========
Training...
  Batch    40  of    268.    Elapsed: 0:02:08.
  

In [547]:
print('Positive samples: %d of %d (%.2f%%)' % (df.label.sum(), len(df.label), (df.label.sum() / len(df.label) * 100.0)))

Positive samples: 6023 of 8551 (70.44%)


In [546]:
# train시 에폭마다 loss가 어떻게 감소하는지 살펴보자.
# import plotly.express as px

# f = pd.DataFrame(loss_values)
# f.columns=['Loss']
# fig = px.line(f, x=f.index, y=f.Loss)
# fig.update_layout(title='Training loss of the Model',
#                    xaxis_title='Epoch',
#                    yaxis_title='Loss')
#fig.show()

## 4. model 저장

In [550]:
torch.save(model.state_dict(), 'GLUE_CoLA_huggingface_BERT_20201022.pt')

In [553]:
model.load_state_dict(torch.load('GLUE_CoLA_huggingface_BERT_20201022.pt'))
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [556]:
minioClient.fput_object('model', 'GLUE_CoLA_huggingface_BERT_20201022.pt', './GLUE_CoLA_huggingface_BERT_20201022.pt')

('c5021ae6aee65fb047d5786363fcd0ec-84', None)

## 5. kubflow fairing을 활용한 학습 수행 및 모델 artifact minio에 저장

In [ ]:
import os

if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils
        from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource

        s3_endpoint = 'nlp-utils:32900'
        minio_endpoint = "http://" + s3_endpoint
        minio_username = "k8s"
        minio_key = "nlplab2020"
        minio_region = "glue-cola-train"

        minio_context_source = MinioContextSource(
            endpoint_url=minio_endpoint,
            minio_secret=minio_username,
            minio_secret_key=minio_key,
            region_name=minio_region
        )

        NAMESPACE = 'workspace'
        IMAGE_NAME = 'fairing-job'
        DOCKER_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'
        BASE_IMAGE = DOCKER_REGISTRY + '/jupyter:tf2.3.0-cpu'

        fairing.config.set_builder(
            'cluster',
            image_name=IMAGE_NAME,
            base_image=BASE_IMAGE,
            context_source=minio_context_source,
            registry=DOCKER_REGISTRY,
            push=True
        )
        
        # cpu 1, memory 1GiB
        fairing.config.set_deployer(
            'job',
            namespace=NAMESPACE,
            pod_spec_mutators=[
                k8s_utils.get_resource_mutator(cpu=1, memory=1)
            ]
        )

        # python3
        # fairing.config.set_preprocessor('python', input_files=[__file__])
        fairing.config.run()
    else:
        remote_train = MyModel()
        remote_train.train()


[I 201022 15:48:53 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7f092df13ac8>
[I 201022 15:48:53 config:136] Using builder: <kubeflow.fairing.builders.cluster.cluster.ClusterBuilder object at 0x7f092df48710>
[I 201022 15:48:53 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f092e03ab38>
[I 201022 15:48:53 cluster:46] Building image using cluster builder.
[I 201022 15:48:53 base:107] Creating docker context: /tmp/fairing_context_73iku4bj
[I 201022 15:48:53 converted_notebook:127] Converting 03.fairing-cola.ipynb to 03.fairing-cola.py
[W 201022 15:48:53 manager:298] Waiting for fairing-builder-j499m-l2fwc to start...
[W 201022 15:48:53 manager:298] Waiting for fairing-builder-j499m-l2fwc to start...
[W 201022 15:48:53 manager:298] Waiting for fairing-builder-j499m-l2fwc to start...
